In [3]:
import torch
import torch.nn as nn
import numpy as np
from normalizer import normalize
import torchtext

/home/mehedi/anaconda3/envs/hf_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from torchtext.data.utils import get_tokenizer

In [6]:
tokenizer=get_tokenizer([normalize)

In [7]:
tokenizer("অফিসের নতুন বড়কর্তা কাজের ব্যাপারে খুব কড়া। কাউকে একবিন্দু ছাড় দেন না।")

'অফিসের নতুন বড়কর্তা কাজের ব্যাপারে খুব কড়া। কাউকে একবিন্দু ছাড় দেন না।'

In [81]:
from torch import math
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position=torch.arange(max_len).unsqueeze(1)
        div_term=1/(10000**(torch.arange(0,d_model,2)/d_model))
        pe=torch.zeros(max_len,d_model)
        pe[:,0::2]=torch.sin(position*div_term)
        pe[:,1::2]=torch.cos(position*div_term)
        self.register_buffer('pe', pe)
    def forward(self,patches):
        return self.dropout(patches+self.pe)




class ImgPatcher(nn.Module):
    def __init__(self,n_patches:int,emb_d:int,channels_first:bool,chw:tuple=):
        super(ImgPatcher,self).__init__()
        self.channels_first=channels_first
        self.N=p
        self.embedding=nn.Linear(255,emb_d)                            #p^2 patches will be created from each image
        self.posenc=None
    
    def forward(self,img):
        H,W,C=img.shape
        if self.channels_first:
            C,H,W=img.shape
        N=(H*W)//(self.p**2)
        remain_pix=(H*W)-(N*self.p**2)
        print(remain_pix)
        flattened_img=img.reshape(N,(self.p*self.p+remain_pix)*C).long()
        if not self.posenc:
            self.posenc=PositionalEncoding(d_model=self.embedding.weight.shape[1],max_len=flattened_img.shape[1])
            print("posenc_generated")
        patches_embedded=self.embedding(flattened_img)
        return self.posenc(patches_embedded)
      


In [86]:
patcher=ImgPatcher(p=16,emb_d=512,channels_first=False)

In [87]:
patcher(torch.randint(0,255,(256,256,3),dtype=torch.uint8).long()).shape

0
posenc_generated


torch.Size([256, 768, 512])

In [73]:
(512*512)/(128*128)

16.0

In [56]:
class ABC:
    def __init__(self):
        self.p=None
    def foo(self):
        if not self.p:
            self.p=1
            return "works"
        else:
            return "works2"
        

In [64]:
abc=ABC()
print(abc.p)

None


In [65]:
print(abc.foo())

works
None


In [66]:
abc.p

1

In [ ]:
from torch import math
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position=torch.arange(max_len).unsqueeze(1)
        div_term=1/(10000**(torch.arange(0,d_model,2)/d_model))
        pe=torch.zeros(max_len,d_model)
        pe[:,0::2]=torch.sin(position*div_term)
        pe[:,1::2]=torch.cos(position*div_term)
        self.register_buffer('pe', pe)
    def forward(self,patches):
        return self.dropout(patches+self.pe)
    



        
        

In [55]:
div_term=10000**(torch.arange(0,768,2)/512)

In [57]:

position=torch.arange(768).unsqueeze(1)

In [56]:
div_term.shape

torch.Size([384])

In [3]:
import torch
from torch import math
d_model=512
max_len=25
    position=torch.arange(max_len).unsqueeze(1)
    div_term=1/(10000**(torch.arange(0,d_model,2)/d_model))
    pe=torch.zeros(max_len,d_model)
    pe[:,0::2]=torch.sin(position*div_term)
    pe[:,1::2]=torch.cos(position*div_term)


In [27]:
position=torch.arange(max_len).unsqueeze(1)
div_term=1/(10000**(torch.arange(0,d_model,2)/d_model))
pe=torch.zeros(max_len,d_model)
pe[:,0::2]=torch.sin(position*div_term)
pe[:,1::2]=torch.cos(position*div_term)
self.register_buffer("pe",pe)

tensor(True)

torch.Size([25, 256])

In [73]:
from normalizer import normalize

ModuleNotFoundError: No module named 'normalizer'

In [135]:
from transformers import ElectraTokenizer

In [138]:
tokenizer=ElectraTokenizer.from_pretrained("csebuetnlp/banglabert")

In [144]:
from transformers import AutoTokenizer, ElectraModelfor{re}
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer
import torch

model = ElectraModel.from_pretrained("csebuetnlp/banglabert")
# tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")

Some weights of the model checkpoint at csebuetnlp/banglabert were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [147]:
model(tokens,attn).last_hidden_state.shape

torch.Size([1, 22, 768])

In [141]:
tokens=tokenizer(fake_sentence,return_tensors='pt')['input_ids']
attn=tokenizer(fake_sentence,return_tensors='pt')['attention_mask']

In [127]:
original_sentence = "আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু করেছেন।"
fake_sentence =["আমি হতাশ কারণ আপনি আমার জন্য অনেক কিছু করেছেন।আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু করেছেন।"]



In [132]:
tkn.unsqueeze(0).shape

torch.Size([1, 1, 22])

In [133]:
model(tkn,attention_mask=attn).logits.shape

torch.Size([1, 22])

In [100]:
test=[original_sentence]*5

In [90]:

tokens=tokenizer.encode(test,return_tensors='pt')
tokenizer.decode(tokens[0])

'[CLS] আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু করেছেন । আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু করেছেন । আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু করেছেন । আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু করেছেন । আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু করেছেন । [SEP]'

In [87]:
test

'আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু করেছেন।আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু করেছেন।আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু করেছেন।আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু করেছেন।আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু করেছেন।'

In [85]:
tokenizer.decode(token_ids=tokens)

TypeError: argument 'ids': 'list' object cannot be interpreted as an integer

In [86]:
tokens

tensor([[   2,  857, 7576, 1155, 1345,  878,  900, 1011,  916, 1403,  205,  857,
         7576, 1155, 1345,  878,  900, 1011,  916, 1403,  205,  857, 7576, 1155,
         1345,  878,  900, 1011,  916, 1403,  205,  857, 7576, 1155, 1345,  878,
          900, 1011,  916, 1403,  205,  857, 7576, 1155, 1345,  878,  900, 1011,
          916, 1403,  205,    3]])

In [66]:
from transformers import pipeline
translate=pipeline.(task="translation",model='saiful9379/Bangla-to-English-Translation')

AttributeError: 'function' object has no attribute 'from_pretrained'

In [149]:
AutoTokenizer.from_pretrained('bert')

0

In [ ]:
class bn_eng_dataset(Dataset):
    def __init__(self,data_dir,bn_tokenizer,eng_tokenizer):
        super(bn_dataset,self).__init__()
        self.data=read_txt(data_dir)
        self.bn_tok=bn_tokenizer
        self.en_tok=eng_tokenizer
        self.bn_tok
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        sample=self.data[idx]
        eng,bn=sample.split('\t')[:2]
        bn=normalize(bn)
        eng=
        

        
        
        
        
        

In [24]:
def read_txt(dir_):
    with open(dir_,'r') as file:
        lines=file.read().strip().split("\n")
        return lines

In [34]:
lines=read_txt("/home/mehedi/Desktop/raghib/CHATBOT_LANGCHAIN_OPENAI/ben.txt")

In [48]:
tokenizer.vocab['[EOS]']=len(tokenizer.vocab)

In [52]:
tokenizer.decode()

AttributeError: 'ElectraTokenizerFast' object has no attribute 'special_tokens'

In [96]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer

model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5_nmt_bn_en")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5_nmt_bn_en", use_fast=False)

KeyboardInterrupt: 